In [ ]:
import mne
from mne.io import concatenate_raws

# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = 1
event_codes = [5, 6, 9, 10, 13, 14]

# This will download the files if you don't have them yet,
# and then return the paths to the files.
physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True, stim_channel='auto',
                             verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events, _ = mne.events_from_annotations(raw)

# Use only EEG channels
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False,
                                  eog=False,
                                  exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, dict(hands=2, feet=3), tmin=1, tmax=4.1,
                     proj=False, picks=eeg_channel_inds,
                     baseline=None, preload=True)

In [ ]:
import numpy as np
from braindecode.datautil.signal_target import SignalAndTarget
from braindecode.datautil.splitters import split_into_two_sets
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:, 2] - 2).astype(np.int64)  # 2,3 -> 0,1

train_set = SignalAndTarget(X[:60], y=y[:60])
test_set = SignalAndTarget(X[60:], y=y[60:])

# train_set, valid_set = split_into_two_sets(train_set,
#                                            first_set_fraction=0.8)
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)

# This will determine how many crops are processed in parallel
input_time_length = 450
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length determines the size of the receptive field of the ConvNet
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=input_time_length,
                        final_conv_length=12).create_network()
to_dense_prediction_model(model)

if cuda:
    model.cuda()


In [ ]:
from torch import optim

optimizer = optim.Adam(model.parameters())

from braindecode.torch_ext.util import np_to_var
# determine output size
test_input = np_to_var(
    np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
if cuda:
    test_input = test_input.cuda()
out = model(test_input)
n_preds_per_input = out.cpu().data.numpy().shape[2]
print("{:d} predictions per input/trial".format(n_preds_per_input))


In [ ]:
from braindecode.experiments.experiment import Experiment
from braindecode.datautil.iterators import CropsFromTrialsIterator
from braindecode.experiments.monitors import RuntimeMonitor, LossMonitor, \
    CroppedTrialMisclassMonitor, MisclassMonitor
from braindecode.experiments.stopcriteria import MaxEpochs
import torch.nn.functional as F
import torch as th
from braindecode.torch_ext.modules import Expression

In [ ]:
# Iterator is used to iterate over datasets both for training
# and evaluation
iterator = CropsFromTrialsIterator(batch_size=32,
                                   input_time_length=input_time_length,
                                   n_preds_per_input=n_preds_per_input)

# Loss function takes predictions as they come out of the network and the targets
# and returns a loss
class LossFunction:
    def __call__(self, preds, targets):
        return F.nll_loss(th.mean(preds, dim=2, keepdim=False), targets)
# loss_function = lambda preds, targets: F.nll_loss(
#     th.mean(preds, dim=2, keepdim=False), targets)

In [ ]:
# Could be used to apply some constraint on the models, then should be object
# with apply method that accepts a module
model_constraint = None
# Monitors log the training progress
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedTrialMisclassMonitor(input_time_length),
            RuntimeMonitor(), ]
# Stop criterion determines when the first stop happens
stop_criterion = MaxEpochs(4)
exp = Experiment(model, train_set, valid_set, test_set, iterator,
                 loss_function, optimizer, model_constraint,
                 monitors, stop_criterion,
                 remember_best_column='valid_misclass',
                 run_after_early_stop=True, batch_modifier=None, cuda=cuda)


In [ ]:
from skorch.classifier import NeuralNet
from skorch.dataset import CVSplit

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class EEGDataSet(Dataset):
    def __init__(self, X, y):
        self.X = X.reshape(-1, 64, 497, 1)
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
from skorch.callbacks import EpochScoring

In [ ]:
train_set = EEGDataSet(train_set.X, train_set.y)
valid_set = EEGDataSet(valid_set.X, valid_set.y)
test_set = EEGDataSet(test_set.X, test_set.y)

In [ ]:
def my_score(net, X=None, y=None):
    losses = net.history[-1, 'batches', :, 'my_score']
    batch_sizes = net.history[-1, 'batches', :, 'valid_batch_size']
    return np.average(losses, weights=batch_sizes)

In [ ]:
skorch_model = NeuralNet(model, LossFunction, optim.Adam, train_split=CVSplit(cv=0.5))
#, callbacks=[('accuracy', EpochScoring('accuracy', lower_is_better=False))])

In [ ]:
skorch_model.fit(train_set)

In [ ]:
F.softmax(torch.Tensor(np.mean(skorch_model.predict(test_set.X), axis=2, keepdims=False)), dim=1).numpy()

In [ ]:
F.softmax(torch.Tensor(np.mean(skorch_model.predict(test_set.X), axis=2, keepdims=False)), dim=1).numpy()

In [ ]:
F.softmax(torch.Tensor(np.mean(skorch_model.predict(test_set.X), axis=2, keepdims=False)[:, 0]))

In [ ]:
F.sigmoid(torch.Tensor([1]))

In [ ]:
skorch_model.history[-1]